You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Outline:
I. Introduction
    A. Definition of Artificial Intelligence
    B. Importance and impact of AI in various industries
II. Latest Trends in Artificial Intelligence
    A. Machine learning and deep learning advancements
    B. Natural language processing developments
    C. AI ethics and regulations
III. Key Players in the AI Industry
    A. Google
    B. IBM
    C. Microsoft
IV. Noteworthy News in Arti

I now can give a great answer
Final Answer: 

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) is a rapidly evolving field that involves the development of computer systems capable of performing tasks that typically require human intelligence. AI has significantly impacted various industries, revolutionizing processes and enhancing efficiency. From healthcare to finance, AI has become an integral part of modern businesses, driving innovation and creating new opportunities for growth.

## Latest Trends in Artificial Intelligence

### Machine learning and deep learning advancements
One of the most significant trends in AI is the continuous advancements in machine learning and deep learning technologies. These technologies enable computers to learn from data and improve their performance over time, leading to more accurate predictions and decision-making processes.

### Natural language processing developments
Another ke

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) is a rapidly evolving field that involves the development of computer systems capable of performing tasks that typically require human intelligence. AI has significantly impacted various industries, revolutionizing processes and enhancing efficiency. From healthcare to finance, AI has become an integral part of modern businesses, driving innovation and creating new opportunities for growth.

## Latest Trends in Artificial Intelligence

### Machine learning and deep learning advancements
One of the most significant trends in AI is the continuous advancements in machine learning and deep learning technologies. These technologies enable computers to learn from data and improve their performance over time, leading to more accurate predictions and decision-making processes.

### Natural language processing developments
Another key trend in AI is the progress in natural language processing (NLP) capabilities. NLP allows computers to understand, interpret, and generate human language, enabling applications such as chatbots, language translation, and sentiment analysis.

### AI ethics and regulations
As AI technologies become more prevalent, the importance of ethical considerations and regulations surrounding AI use has come to the forefront. Issues such as data privacy, bias in algorithms, and accountability are crucial aspects that need to be addressed to ensure the responsible development and deployment of AI systems.

## Key Players in the AI Industry

### Google
Google is a major player in the AI industry, with its research and development in machine learning, natural language processing, and computer vision. Google's AI technologies power various products and services, including Google Search, Google Assistant, and Google Cloud AI.

### IBM
IBM is renowned for its AI research and solutions, particularly in areas such as healthcare, finance, and supply chain management. IBM's AI platform, Watson, is widely used for data analysis, predictive modeling, and decision support.

### Microsoft
Microsoft has made significant investments in AI, integrating AI capabilities into its products and services, such as Azure AI and Microsoft Cognitive Services. Microsoft's AI initiatives focus on empowering businesses with AI-driven solutions for improved productivity and decision-making.

## Noteworthy News in Artificial Intelligence

### Recent breakthroughs in AI research
Recent breakthroughs in AI research have paved the way for new advancements in areas like reinforcement learning, generative models, and explainable AI. These breakthroughs have the potential to transform industries and drive further innovation in AI development.

### AI applications in healthcare and finance
AI applications in healthcare and finance have gained traction, with AI-powered solutions for disease diagnosis, personalized treatment plans, fraud detection, and risk management. These applications have the potential to improve outcomes, reduce costs, and enhance decision-making processes in these sectors.

## Conclusion

As AI continues to evolve and shape the future of industries, staying informed about the latest trends and key players in the AI industry is crucial for businesses and professionals. By exploring the advancements in machine learning, NLP, and AI ethics, businesses can leverage AI technologies to drive innovation and achieve competitive advantage. The collaboration between industry leaders like Google, IBM, and Microsoft, along with breakthroughs in AI research, will further propel the growth and impact of AI in various sectors. It is essential for tech enthusiasts and business professionals to stay updated on AI news and innovations to harness the full potential of AI solutions for their organizations.

## Call to Action

Stay updated on AI news and innovations by following reputable sources such as Forbes, Wired, and TechCrunch. Explore AI solutions for your businesses to unlock new opportunities and drive growth through the power of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence and its impact on various industries
- Introduction to the latest trends, key players, and noteworthy news in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- Automation in various industries
- Ethical considerations in AI development

2. Key Players in the AI Industry


I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has transformed various industries, from healthcare to finance, by automating processes, analyzing extensive data sets, and making decisions with minimal human intervention. The significant impact of AI is reshaping how businesses function and altering our interaction with technology. This blog post delves into the latest trends, key players, and noteworthy news in the AI industry, offering valuable insights to business professionals, technology enthusiasts, and individuals intrigued by the future of AI.

## Latest Trends in Artificial Intelligence

One of the most notable trends in AI is the progress in machine learning, where algorithms are trained to enhance their performance on specific tasks through experience. This advancement has resulted in breakthroughs in fields such as image recognition, natural language processing, a

In [16]:
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has transformed various industries, from healthcare to finance, by automating processes, analyzing extensive data sets, and making decisions with minimal human intervention. The significant impact of AI is reshaping how businesses function and altering our interaction with technology. This blog post delves into the latest trends, key players, and noteworthy news in the AI industry, offering valuable insights to business professionals, technology enthusiasts, and individuals intrigued by the future of AI.

## Latest Trends in Artificial Intelligence

One of the most notable trends in AI is the progress in machine learning, where algorithms are trained to enhance their performance on specific tasks through experience. This advancement has resulted in breakthroughs in fields such as image recognition, natural language processing, and predictive analytics. Moreover, developments in natural language processing have empowered AI systems to comprehend and generate human language, unlocking new possibilities in virtual assistants, chatbots, and translation services. Automation across various industries is another pivotal trend, with AI-driven robots and software optimizing processes and boosting efficiency in manufacturing, healthcare, and customer service. Nevertheless, these advancements bring about ethical considerations in AI development, including algorithmic bias, data privacy concerns, and the impact on job displacement.

## Key Players in the AI Industry

Numerous tech giants and innovative startups are at the forefront of the AI industry, spearheading research, development, and adoption of AI technologies. Companies like Google, Amazon, Microsoft, IBM, Tesla, and OpenAI are heavily investing in AI research and applications, pushing boundaries in artificial intelligence. Google utilizes deep learning algorithms to power products like Google Search and Google Photos, while Amazon's AI-driven recommendations enhance the shopping experience for millions of customers. Microsoft's Azure platform offers a wide array of AI services for businesses, and IBM's Watson platform is transforming industries with its cognitive computing capabilities. Tesla's self-driving technology and OpenAI's research in artificial general intelligence are also instrumental in shaping the future of AI.

## Noteworthy News in Artificial Intelligence

Recent advancements in AI have showcased promising applications in healthcare, with AI-powered systems diagnosing diseases, personalizing treatment plans, and enhancing patient outcomes. In finance and banking, AI is utilized for fraud detection, risk assessment, and algorithmic trading, improving efficiency and security in the sector. AI is also making significant strides in autonomous vehicles, with companies like Tesla and Waymo leading the charge in developing self-driving cars that could revolutionize transportation. It is crucial to monitor regulatory updates in AI technology as governments and organizations work to establish guidelines and standards for the responsible development and deployment of AI systems.

In conclusion, the future of artificial intelligence presents exciting possibilities and challenges. By staying informed about the latest trends, key players, and noteworthy news in the AI industry, businesses, technology enthusiasts, and individuals can make informed decisions on leveraging AI technology for innovation and growth. As AI continues to advance and shape our world, it is essential to consider the ethical implications and societal impact of these advancements, ensuring that AI benefits humanity as a whole. Stay tuned for more updates on the ever-evolving landscape of artificial intelligence. 

---

This blog post on Artificial Intelligence covers the latest trends, key players, and noteworthy news in the AI industry, providing valuable insights to a diverse audience interested in the future of AI technology. By incorporating SEO keywords naturally and structuring the post with engaging sections and subtitles, the content is tailored to attract and inform business professionals, technology enthusiasts, and individuals curious about the impact of AI on society. Proofread for grammatical errors and alignment with the brand's voice, this blog post is ready for publication and aims to educate and engage readers on the dynamic world of artificial intelligence.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).